In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from scipy import stats
from utils.post_processing_utils import open_one_experiment
from utils.value_change_utils import CustomAlignedDataRewardBlocks, get_all_experimental_records, get_block_change_info, add_traces_and_peaks, plot_mean_trace_for_condition_value_switch, one_session_get_block_changes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
root_directory = 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis\\utils\\'
processed_data_dir = os.path.join(root_directory, 'value_change_data')
block_data_file = os.path.join(processed_data_dir, 'value_switch_all_tail_mice_test.csv')
all_reward_block_data = pd.read_pickle(block_data_file)

In [ ]:
#processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\value_switch_data')
#block_data_file = os.path.join(processed_data_dir, 'value_switch_all_tail_mice.csv')

#all_reward_block_data = pd.read_pickle(block_data_file)

In [53]:
all_reward_block_data['session'].unique()

array(['20210518', '20210519', '20210520', '20210521', '20210608',
       '20210609', '20210610', '20210611'], dtype=object)

In [54]:
sorted_data = all_reward_block_data.sort_values(['mouse', 'session', 'trial number']).reset_index(drop=True)
sorted_data['block switches'] = sorted_data['block number'] - sorted_data['block number'].shift()
sorted_data['new sessions'] = sorted_data['session'].ne(sorted_data['session'].shift().bfill()).astype(int)
sorted_data.iloc[0, sorted_data.columns.get_loc('new sessions')] = 1
sorted_data.loc[sorted_data['new sessions'] == 1, 'block switches'] = 1
block_switch_inds = sorted_data.loc[sorted_data['block switches'] != 0].reset_index(drop=True)

In [55]:
block_switch_inds

,block number,trial number,peak size,relative reward amount,session,mouse,contra reward amount,traces,time points,block switches,new sessions
0,0.0,1.0,0.202167,0.0,20210518,SNL_photo37,2.0,"[1.0352594779009694, 1.0352401281347636, 1.035...","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,1
1,1.0,100.0,0.706991,4.0,20210518,SNL_photo37,6.0,"[-0.8428589943684968, -0.8422359897595617, -0....","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,0
2,0.0,1.0,0.183061,0.0,20210519,SNL_photo37,2.0,"[-0.34049675343950536, -0.34100296900505234, -...","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,1
3,1.0,100.0,-0.294811,4.0,20210519,SNL_photo37,6.0,"[-1.1854710372872375, -1.1856475461142402, -1....","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,0
4,0.0,0.0,1.687711,0.0,20210520,SNL_photo37,2.0,"[-1.1998684566840374, -1.1995577396070998, -1....","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,1
5,1.0,100.0,0.180263,4.0,20210520,SNL_photo37,6.0,"[-0.242590064852729, -0.24177875532048065, -0....","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,0
6,0.0,4.0,0.820485,0.0,20210521,SNL_photo37,2.0,"[0.5823024879985481, 0.5823106224925365, 0.582...","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,1
7,5.0,102.0,1.460199,-4.0,20210521,SNL_photo37,2.0,"[-0.31709403269580533, -0.31694960908338454, -...","[-8.0, -7.999899999374996, -7.999799998749992,...",5.0,0
8,0.0,7.0,0.753964,0.0,20210518,SNL_photo43,2.0,"[-0.24968602526713127, -0.24988176450988966, -...","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,1
9,1.0,103.0,0.019239,4.0,20210518,SNL_photo43,6.0,"[1.5092551352316281, 1.5089126623322853, 1.508...","[-8.0, -7.999899999374996, -7.999799998749992,...",1.0,0


In [71]:
num_trials_to_look_at = 50
min_num_trials = 70
traces = []
peaks = []
trial_nums = []
block_inds = []
rel_reward_amounts = []
reward_amounts = []
mouse_ids = []
session_ids = []
num_blocks = block_switch_inds.index.values.shape[0]
for block_num, block in block_switch_inds.iterrows():
    first_trial = block['trial number']
    mouse = block['mouse']
    session = block['session']
    block_id = block['block number']
    block_switch_trial_num = block['trial number']
    all_session_trials = sorted_data[(sorted_data['mouse'] == mouse) & (sorted_data['session'] == session)]
    all_block_trials = all_session_trials[all_session_trials['block number'] == block_id]
    if mouse == 'SNL_photo44':
        print(all_block_trials.shape[0])
    if block_id == 0 or all_block_trials.shape[0]>=min_num_trials:
        print(mouse)
        last_trials_of_block = all_block_trials[-num_trials_to_look_at:]
        mean_trace = last_trials_of_block['traces'].apply(np.mean)
        avg_trace = last_trials_of_block.groupby(['mouse', 'contra reward amount'])['traces'].apply(np.mean).values[0]
        decimated = decimate(avg_trace[int(len(avg_trace)/2):], 10) 
        peak_idx = peakutils.indexes(decimated)[0] 
        peak = decimated[peak_idx]
        traces.append(decimated)
        peaks.append(peak)
        trial_nums.append(last_trials_of_block['trial number'].values)
        rel_reward_amounts.append(last_trials_of_block['relative reward amount'].values[0])
        reward_amounts.append(last_trials_of_block['contra reward amount'].values[0])
        block_inds.append(last_trials_of_block.index.values[0])
        mouse_ids.append(mouse)
        session_ids.append(session)

SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo37
SNL_photo43
SNL_photo43
SNL_photo43
SNL_photo43
SNL_photo43
SNL_photo43
SNL_photo43
SNL_photo43
49
SNL_photo44
69
39
SNL_photo44
107
SNL_photo44
40
SNL_photo44
90
SNL_photo44
45
SNL_photo44
85
SNL_photo44


In [74]:
avg_block_data = {}
avg_block_data['block id'] = block_inds
avg_block_data['peaks'] = peaks
avg_block_data['relative reward amount'] = rel_reward_amounts
avg_block_data['contra reward amount'] = reward_amounts
avg_block_data['mouse'] = mouse_ids
avg_block_data['session'] = session_ids
avg_block_dataf = pd.DataFrame(avg_block_data)

In [75]:
avg_block_dataf['avg traces'] = pd.Series(traces, index=avg_block_dataf.index)
df_for_plot = avg_block_dataf.groupby(['mouse', 'session', 'relative reward amount', 'contra reward amount'])['peaks'].apply(np.mean)
df_for_plot = df_for_plot.reset_index()

In [76]:
mice = []
reward_changes = []
cue_changes = []
session_ids = []

first_blocks = df_for_plot[df_for_plot['relative reward amount'] == 0]
for first_block_ind, first_block in first_blocks.iterrows():
    mouse = first_block['mouse']
    session = first_block['session']
    session_blocks = df_for_plot[np.logical_and(df_for_plot['mouse'] == mouse, df_for_plot['session'] == session)]
    other_block = session_blocks.loc[session_blocks['relative reward amount'] != 0]
    if other_block.shape[0] > 0:
        mice.append(mouse)
        reward_changes.append(other_block['relative reward amount'].values[0])
        cue_change = (other_block['peaks'] - first_block['peaks']).values[0]
        cue_changes.append(cue_change)
        session_ids.append(session)
diff_block_data = {}
diff_block_data['mouse'] = mice
diff_block_data['reward size change'] = reward_changes
diff_block_data['cue size change'] = cue_changes
diff_block_data['session'] = session_ids
diff_block_dataf = pd.DataFrame(diff_block_data)

In [78]:
df_for_plot1 = diff_block_dataf.groupby(['mouse', 'reward size change'])['cue size change'].apply(np.mean)
df_for_plot1 = df_for_plot1.reset_index()

In [79]:
df_for_plot1

,mouse,reward size change,cue size change
0,SNL_photo37,-4.0,-0.628180
1,SNL_photo37,4.0,-0.115648
2,SNL_photo43,-4.0,0.073748
3,SNL_photo43,4.0,0.037211
4,SNL_photo44,-4.0,-0.328067
5,SNL_photo44,4.0,-0.470801


In [80]:
df1 = df_for_plot1.pivot(index='reward size change', columns='mouse', values='cue size change').sort_values('reward size change', ascending=False)

In [81]:
df1

mouse,SNL_photo37,SNL_photo43,SNL_photo44
reward size change,,,
4.0,-0.115648,0.037211,-0.470801
-4.0,-0.628180,0.073748,-0.328067


In [82]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    
    fg = sns.FacetGrid(data=data, hue='mouse', aspect=1.61)
    fg.map(plt.scatter, 'reward size change', 'cue size change', s=6)



In [83]:
fg = sns.FacetGrid(data=df_for_plot1, hue='mouse', aspect=0.8, palette='viridis')
fg.map(plt.scatter, 'reward size change', 'cue size change', s=20)
ax=fg.axes[0][0]
ax.axhline(0, color='k', lw=0.5)
ax.set_xticks([-4, 4])
ax.set_xlabel(r'relative value ($\mu$l)')
ax.set_xlim([-6, 6])
plt.ylabel(r'$\Delta$ z-score')
plt.tight_layout()


In [52]:
fg.axes

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000000B0B9C1D0>]],
      dtype=object)